# Apriori Approach to determine Frequent Item Sets

In [ ]:
#Import libraries
from timeit import default_timer as timer
from itertools import combinations
from collections import Counter
from itertools import chain
import pandas as pd
import numpy as np
import csv

#Final frequent sets declaration
n_frequent_dict = {}
apriori_dict = {}
apriori_frequent_dict = {}

#Read from CSV file into DataFrame
all_columns = ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15"]
df = pd.read_csv("Z:\Big Data and Data Mining\Assignments\CourseWork1\GroceryStore.csv",names = all_columns)
df.head()

In [ ]:
#open file to write the intermediate updates
output_file = open("Z:\Big Data and Data Mining\Assignments\CourseWork1\Apriori.txt",'w')

In [ ]:
#Read minimum support and confidence
min_support = input("Enter the value of minimum support\n")
min_confidence = input("Enter the value of minimum confidence\n")

In [ ]:
#Find unique_items in list
def unique_items(dataframe):
    item_list = dataframe.values.tolist()
    flat_item_list = list(chain.from_iterable(item_list))
    unique_item_list = list(set(flat_item_list))
    unique_item_list = list(filter(None, unique_item_list))
    return unique_item_list    

In [ ]:
#Function to write list into file and print it on screen
def write_list_to_file(message,item_list):
    item_list_str = "\n".join(item_list)
    output_file.write(message + item_list_str)
    print(message + item_list_str)

In [ ]:
#support count of candidates
def find_support(n,item_list,data_list):
    k = 0
    count = 0
    support = [0]*len(item_list)
    item = [0]*len(item_list)
    for i in item_list:
        for j in data_list:
            data_set = set(j)
            if(i.issubset(data_set)):
                  count  +=1
        support[k] = count
        item[k] = i
        apriori_dict[i] = count
        k+=1
        count = 0
    return support,item

In [ ]:
#Find candidates
def combine(frequent_set,n):
    combined_set = set()
    for x in frequent_set:
        i = set(x)
        for y in frequent_set:
            j = set(y)
            if(len(i.union(j)) == n):
                combined_temp = [x.union(y)]
                combined_set = combined_set.union(combined_temp)
    return combined_set

In [ ]:
#Pruning candidates to find frequent item sets
def prune_candidates(min_support,support,item):
    frequent_itemset = set()
    infrequent_itemset = set()
    frequent_count = []
    for i in range (len(item)):
        if(support[i] >= int(min_support)):
            frequent_itemset.add(item[i])
            frequent_count.append(support[i])
        else:
            infrequent_itemset.add(item[i])
    return frequent_itemset,infrequent_itemset,frequent_count

In [ ]:
#candidate set generation
def candidate_set(combination_set,infrequent_set):
    candidates = combination_set.copy()
    for i in combination_set:
        for j in infrequent_set:
            if(j.issubset(i)):
                candidates.remove(i)
                break
    return candidates

In [ ]:
#save the built frequent sets
def store(freq_set,freq_count,n):
    frequent_dict ={}
    for i,j in zip(freq_set,range(len(freq_count))):   
        n_frequent_dict[i] = freq_count[j] 
        frequent_dict[i] = freq_count[j] 
    if(len(frequent_dict) >= 1):
        item_set_sel = "{0}_item_sets".format(n)
        apriori_frequent_dict[item_set_sel] = frequent_dict

In [ ]:
#Apriori Algorithm
def apriori(dataframe,n):
    
    start = timer()
    #Find unique items
    unique_item_list = unique_items(dataframe)
    item_list = [x for x in unique_item_list if pd.isnull(x) == False]
    data_list = df.values.tolist() 
    
    #unique frozen set
    item_set = set()
    for i in item_list:
        if i:
            item_set.add(frozenset([i]))
            
    #Find candidates,support and 1 frequent set
    
    count,item = find_support(1,item_set,data_list)  
    frequent_set,infrequent_set,frequent_count = prune_candidates(min_support,count,item)
    store(frequent_set,frequent_count,1)
        
    #     Find 2-n frequent itemsets
    #     Step1 : Find possible combinations
    #     Step2 : Find candidates
    #     Step3 : Find support count
    #     Step4 : Prune candidates that donot satisfy min support condition to generate frequent itemsets
    #     Step5 : Save the result into a dictionary for easy access
    
    for i in range(2,n):
        combination_set = combine(frequent_set,i)
        candidates = candidate_set(combination_set,infrequent_set)
        count,item = find_support(i,candidates,data_list)
        frequent_set,infrequent_set,frequent_count = prune_candidates(min_support,count,item)
        store(frequent_set,frequent_count,i)
    
    end = timer()
    
#     Preparing to write output to file and to print
    i = 0
    frequent_item_str = [0]*len(n_frequent_dict)
    for key, value in n_frequent_dict.items():
        frequent_item_str[i] = str('{}----->{}'.format(list(key), value))
        i = i+1 
        
    #calculate time taken to execute for bruteforce approach
    total_time = end - start
    
    #Write output to file and print the output
    output_file.write("1.minimum support :" + min_support)
    print("1.minimum support:"+min_support)
    output_file.write("\n\n2.minimum confidence :" + min_confidence)
    print("\n\n2.minimum confidence :" + min_confidence)
    output_file.write("\n\n3.Apriori Approach:\n\n") 
    print("\n\n3.Apriori Approach:") 
    write_list_to_file("\n\n3a.Unique item list:\n",item_list)
    write_list_to_file("\n\n\n3b.frequent item sets:\n",frequent_item_str)
    output_file.write("\n\n4.Time taken for execution using Apriori approach:"+str(total_time)) 
    print("\n\n4.Time taken for execution using Apriori approach:" + str(total_time))

In [ ]:
#Apriori function call
apriori(df,6)

In [ ]:
#Function to determine Association Rules
def association_rules():
    min_confidence_value = int(min_confidence)/100
    output_file.write("\n\n5.Association Rules:\n")
    n = len(apriori_frequent_dict)
    for i in range(n,0,-1):
        item_set_sel = "{0}_item_sets".format(i)
        freq_len = len(apriori_frequent_dict[item_set_sel])
        for j in range(0,freq_len):
            test_list = []
            test_item = {}
            test_item = (apriori_frequent_dict[item_set_sel])
            test_list = list(apriori_frequent_dict[item_set_sel].keys())
            for k in range(1,i):
                index = test_list[j]
                numerator = apriori_dict[index]  
                comb = combinations(test_list[j], k)
                for l in comb:
                    z = frozenset(l)
                    h="{0}_item_sets".format(len(l))
                    deno = apriori_dict[z]
                    if((numerator/deno)>=min_confidence_value):
                        print((str(l)+"   -->"+str(set(test_list[j])-set(l)))+"      :"+str(round((numerator/deno)*100,2))+"%")
                        output_file.write(("\n"+str(l)+"   -->"+str(set(test_list[j])-set(l)))+"      :"+str(round((numerator/deno)*100,2))+"%")

In [ ]:
#Association rules function call
association_rules()

In [ ]:
# Close output file after writing
output_file.close()